In this notebook we show the alchemical perturbations from Toluene, to aniline, to phenol. <br>

Using the pcX-2 (H pc-2)/PBE0 level of theory. We start from the optimized geometry of toluene. 

We created as target reference the aniline and the phenol molecules, placing artificial charges at the nuclei of C and H. <br> 
The choice of the basis set ensure a small basis set error: <br>

For aniline is   0.027 mHa   (-287.361445 true basis set, - 287.361472 toluene basis set)

For phenol is slightly higher  2.6 mHa, because of the larger perturbation (-307.21005 true basis set, -307.20742 toluene basis set)

In any case we will compare our results fixing the basis set and geometry to the toluene's ones. <br>




The goal is to comare the goodness of Levy's formula ( was him? ) to the "local" alchemical perturbations up to order 3. <br>
To do so I created two fractional charge molecules whose nuclear charges are half way between the reference (toluene) and the two targets (aniline and phenol). The nuclear charges of the carbon and the two hydrogen atoms are (6.5,.5) and (7,.5,.5)  <br>
The electron density of these intermediate molecules $P$ were calculated at the same DFT level as the reference. Than conracted with the $\Delta V_{ne}$ operator to get the alchemical first order Helmann-Feynman derivatives   

# Results 


| Target | APDFT1  |APDFT2  |APDFT3  |Levy | True |
| :--- | --- | --- | --- |--- | -- | 
| Aniline | -284.9958  | -287.4710 | -287.3423  | -287.38576 | -287.36147 |
| Phenol  | -298.6528 | -307.1101 | -307.2181 | -307.18447 | -307.20742 |

For both molecules the accuracy is better than APDFT2 and comparable to the APDFT3 perturbation, but the error has opposite sign 


In [1]:
# averaging Levy and APDT3 ?
(-287.3423  -287.38576)/2,(-307.2181  -307.18447 )/2 

(-287.36403, -307.201285)

In [4]:
from pyscf import scf,gto
import basis_set_exchange as bse
import numpy as np
import matplotlib.pyplot as plt
import sys
a2c={1:"y",6:"k",7:"b",8:"r"}

In [5]:
sys.path.append("../Alchemical_CPKS_perturbator")
from FcMole import *
from AP_class import APDFT_perturbator as AP

In [6]:
basis_pcx2={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])\
           ,'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])\
           ,'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])}

In [10]:
tol_atoms=gto.fromfile("tol_opt.xyz")
pcx2_tol=gto.M(atom=tol_atoms,basis=basis_pcx2)

In [11]:
mf_tol=scf.RKS(pcx2_tol)
mf_tol.xc="PBE0"
mf_tol.scf()

converged SCF energy = -271.33657051295


-271.33657051294955

In [12]:
an_at_tol=FcM_like(pcx2_tol,fcs=[[0,3 ],[-1,1] ])
fen_at_tol=FcM_like(pcx2_tol,fcs=[[0,1,3 ],[-1,-1,2 ] ])

In [13]:
mf_an_tol=scf.RKS(an_at_tol)
mf_fen_tol=scf.RKS(fen_at_tol)
mf_an_tol.xc="PBE0"
mf_fen_tol.xc="PBE0"

In [14]:
e_an=mf_an_tol.scf()
e_fen=mf_fen_tol.scf()

/home/giorgio/anaconda3/lib/python3.8/site-packages/pyscf/gto/mole.py:1209: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -287.361472480971
converged SCF energy = -307.207424364003


In [15]:
#e_an=mf_an_tol.scf()#nverged SCF energy = -287.36144546195 # with proper basis set

#e_fen=mf_fen_tol.scf()#converged SCF energy = -307.210051602572 


In [16]:
#ABSEs
-287.36144546195+287.36147248097,  -307.210051602572 +307.207424364003

(2.7019020023999474e-05, -0.0026272385690049305)

In [17]:
e_tol=mf_tol.e_tot

In [18]:
#ap_an=AP(mf_an_tol,sites=[0,2])
ap_tol=AP(mf_tol,sites=[0,1,3])

In [19]:
ap_tol.build_all()
ap_tol.APDFT1(np.array([-1,0,1])),ap_tol.APDFT2(np.array([-1,0,1])),ap_tol.APDFT3(np.array([-1,0,1])),e_an

(-284.99588818927543,
 -287.4710810516354,
 -287.34238312295787,
 -287.36147248097086)

In [20]:
ap_tol.APDFT1(np.array([-1,-1,2])),ap_tol.APDFT2(np.array([-1,-1,2])),ap_tol.APDFT3(np.array([-1,-1,2])),e_fen

(-298.65284086385833,
 -307.1101952371722,
 -307.21817433247406,
 -307.2074243640035)

In [21]:
from AP_utils import DeltaV

In [22]:
fc_tf=FcM_like(pcx2_tol,fcs=[[0,1,3 ],[-.5,-.5,1 ]])

In [23]:
mf_fc_tf=scf.RKS(fc_tf)
mf_fc_tf.xc="PBE0"
mf_fc_tf.scf(dm0=(mf_fen_tol.make_rdm1()+mf_tol.make_rdm1())/2)

converged SCF energy = -287.121974882745


-287.12197488274546

In [24]:
dV= DeltaV(pcx2_tol,dL=[[0,1,3 ],[-1,-1,2 ] ])

In [25]:
p0=mf_fen_tol.make_rdm1()
p1=mf_fc_tf.make_rdm1()

In [26]:
#Levy's formula vs True values

np.einsum("ij,ij",p1,dV)+mf_tol.e_tot -mf_tol.energy_nuc()+mf_fen_tol.energy_nuc(),e_fen

(-307.18447221045847, -307.2074243640035)

In [27]:
ap_tol.APDFT1(np.array([-1,-1,2])),ap_tol.APDFT2(np.array([-1,-1,2])),ap_tol.APDFT3(np.array([-1,-1,2])),e_fen

(-298.65284086385833,
 -307.1101952371722,
 -307.21817433247406,
 -307.2074243640035)

Accuracy better than APDFT2

# To aniline 

In [28]:
dV1= DeltaV(pcx2_tol,dL=[[0,3 ],[-1,1] ])

In [29]:
fc_tf1=FcM_like(pcx2_tol,fcs=[[0,3 ],[-.5,.5 ]])
mf_fc_tf1=scf.RKS(fc_tf1)
mf_fc_tf1.xc="PBE0"
mf_fc_tf1.scf(dm0=(mf_an_tol.make_rdm1()+mf_tol.make_rdm1())/2)

converged SCF energy = -278.770769457381


-278.77076945738133

In [30]:
p2=mf_fc_tf1.make_rdm1()

In [31]:
#Levy's formula vs True values
np.einsum("ij,ij",p2,dV1)+mf_tol.e_tot -mf_tol.energy_nuc()+mf_an_tol.energy_nuc(),e_an

(-287.38576443088675, -287.36147248097086)

In [32]:
ap_tol.APDFT1(np.array([-1,0,1])),ap_tol.APDFT2(np.array([-1,0,1])),ap_tol.APDFT3(np.array([-1,0,1])),e_an

(-284.99588818927543,
 -287.4710810516354,
 -287.34238312295787,
 -287.36147248097086)

ACCURACY is better than APDFT2, APPROXIMATELY apdft3